# GRT Final

This is the final development notebook of the GRT project.  In this notebook converge all the lessons learned during development.

In [1]:
import os
NTH=4
os.environ["OMP_NUM_THREADS"]="%d"%NTH

In [2]:
from gravray import *
from gravray.util import *
from gravray.stats import *
from gravray.spice import *
from gravray.plot import *
from gravray.sampling import *
from gravray.orbit import *

import skmonaco
import multiprocessing as mp

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%matplotlib nbagg

## Initialize

In [4]:
dt=elTime(0)
Spice.loadKernels()
cneos=pd.read_csv("data/cneos_fireball_data_location.csv")
cneos.set_index("Name",inplace=True)
earth=Body("EARTH")
NP=int(mp.cpu_count()/NTH)
print("Available number of processors: ",NP)

Available number of processors:  1


In [5]:
#Site 
lon=59.8703**Angle.Deg #rad
lat=55.0958*Angle.Deg #rad
alt=23.3*Const.km #m

#Time
datestring="02/15/2013 03:20:34"
fmt="%m/%d/%Y %H:%M:%S"
t=Spice.str2tdb(datestring)

#Population
weights=[0.6]
locs=[
    [0.5,0.5,-2.0],
    [2.0,0.3,-2.6]
]
scales=[
    [1.3,0.7,0.5],
    [0.4,0.9,1.6]
]
angles=[
    [-40.0*Angle.Deg,-86.0*Angle.Deg,0.0*Angle.Deg],
    [+80.0*Angle.Deg,-109.0*Angle.Deg,0.0*Angle.Deg]
]
ranges=[
    [Util.fin2Inf(1e-2,360.0),Util.fin2Inf(360.0-1e-2,360.0)],
    [Util.fin2Inf(1e-2,360.0),Util.fin2Inf(360.0-1e-2,360.0)]
]
NEOs=MultiVariate([1,1,1,0,0])
NEOs.setUnflatten(weights,locs,scales,angles,ranges)

#Direction
Aloc=103.5*Angle.Deg
hloc=18.55*Angle.Deg
vimp=-18.6*Const.km/Const.s

Rdir=np.array([[Aloc],[hloc],[vimp]])

#Input conditions
print(f"Impact conditions:")
print(f"\tDate: {datestring}")
print(f"\tlon. {lon*Angle.Rad:.4g}, lat. {lat*Angle.Rad:.4g}, alt. {alt/Const.km:.4g} km")
print(f"\tAloc = {Aloc*Angle.Rad:.4g}, hloc = {lat*Angle.Rad:.4g}, vimp = {vimp/Const.km} km/s")

Impact conditions:
	Date: 02/15/2013 03:20:34
	lon. 61.54, lat. 55.1, alt. 23.3 km
	Aloc = 103.5, hloc = 55.1, vimp = -18.6 km/s


## Test integrations

### Three dimensional (Monte Carlo)

In [31]:
prob=GrtProb(t,earth,NEOs,verbose=True)
prob.setLocation(lon,lat,alt)
p=prob.calcPdir([Aloc,hloc,vimp])
p,prob.ph,prob.detJ,prob.Rdir

Impact conditions: np.array([[1.806415775814131],[0.32375857624494814],[-18600.0]])
Terminal elements: [  0.72231578   0.5297087    5.18079402 326.57568287 103.0150748
  24.56944643]
Unbound terminal elements: [ 0.89757661  0.11897496 -3.518794    2.27937803 -0.91414209 -2.61391129]


(5.228572267234881e-26,
 5.586619321805399e-05,
 -9.359098886202953e-22,
 [1.806415775814131, 0.32375857624494814, -18600.0])

In [27]:
NCALLS=0
points=[]
prob=GrtProb(t,earth,NEOs,verbose=False)
prob.setLocation(lon,lat,alt)
def f(X):
    global points,NCALLS
    p=prob.calcPdir(list(X))
    points+=[list(X)+[p]]
    NCALLS+=1
    if (NCALLS%10000)==0:
        print(f"{NCALLS} calls... ",)
        dt=elTime(1)
    return p

In [30]:
print("Starting integration...")
NCALLS=0
N=int(1000/NP)*NP
dt=elTime(0)
integ=skmonaco.mcmiser(f,npoints=N,
                      xl=[0.0,0.0,-72.2*Const.km/Const.s],
                      xu=[2*np.pi,np.pi/2,-11.1*Const.km/Const.s],
                      nprocs=NP)
dt=elTime(1)
print("Three dimensional MISER MC: ",integ,NCALLSF)

Three dimensional MISER MC:  (8.390067184951666e-21, 3.0532176986657898e-21) 0


In [29]:
print("Saving points...")
np.savetxt("points.dat",points)

Saving points...


--End--